In [2]:
import json
import pandas as pd
import os
import numpy as np
import requests
from dotenv import load_dotenv
import pathlib as path

In [3]:
load_dotenv()
api_key = os.getenv("CENSUS_API_KEY")

In [4]:
# API endpoint for listing variables - needed to check if variables that I wasa given by ChatGPT were correct:
url = "https://api.census.gov/data/2022/acs/acs1/variables.json"

# Make the API request
response = requests.get(url)
variables = response.json()

# Print the available variables
#for variable in variables['variables']:
 #   print(variable)

In [5]:
#This API request is for " Mortgage Status by Selected Monthly Owner Costs as a Percentage of Household Income in the Past 12 Months"
#It gives us a sense of how much income is being spent on housing. Might suggest what kind of housing might be appropriate (eg affordable, mid-market, luxury) 

#test_url = f"https://api.census.gov/data/2022/acs/acs1?get=group(B25091)&ucgid=pseudo(0100000US$3100000)" - this was metro data - changing to county below:
test_url = f"https://api.census.gov/data/2022/acs/acs1?get=group(B25091)&for=county:*&in=state:*&key={api_key}"
query = requests.get(test_url).json()
json.dumps(query, indent=4)

'[\n    [\n        "B25091_001E",\n        "B25091_001EA",\n        "B25091_001M",\n        "B25091_001MA",\n        "B25091_002E",\n        "B25091_002EA",\n        "B25091_002M",\n        "B25091_002MA",\n        "B25091_003E",\n        "B25091_003EA",\n        "B25091_003M",\n        "B25091_003MA",\n        "B25091_004E",\n        "B25091_004EA",\n        "B25091_004M",\n        "B25091_004MA",\n        "B25091_005E",\n        "B25091_005EA",\n        "B25091_005M",\n        "B25091_005MA",\n        "B25091_006E",\n        "B25091_006EA",\n        "B25091_006M",\n        "B25091_006MA",\n        "B25091_007E",\n        "B25091_007EA",\n        "B25091_007M",\n        "B25091_007MA",\n        "B25091_008E",\n        "B25091_008EA",\n        "B25091_008M",\n        "B25091_008MA",\n        "B25091_009E",\n        "B25091_009EA",\n        "B25091_009M",\n        "B25091_009MA",\n        "B25091_010E",\n        "B25091_010EA",\n        "B25091_010M",\n        "B25091_010MA",\n        "

In [6]:
#Raw dataframe - column headings are the code that refers to different screening variables. I researched
#these are used the common names in my column rename code below:

housing_percent_income_2022 = pd.DataFrame(query)
housing_percent_income_2022.columns = housing_percent_income_2022.iloc[0]
housing_percent_income_2022 = housing_percent_income_2022[1:]
housing_percent_income_2022.reset_index(drop=True, inplace=True)
housing_percent_income_2022

,B25091_001E,B25091_001EA,B25091_001M,B25091_001MA,B25091_002E,B25091_002EA,B25091_002M,B25091_002MA,B25091_003E,B25091_003EA,...,B25091_022M,B25091_022MA,B25091_023E,B25091_023EA,B25091_023M,B25091_023MA,GEO_ID,NAME,state,county
0,76213,None,4078,None,45042,None,3521,None,4140,None,...,545,None,739,None,654,None,0500000US01003,"Baldwin County, Alabama",01,003
1,32866,None,1865,None,14709,None,1896,None,1392,None,...,533,None,819,None,560,None,0500000US01015,"Calhoun County, Alabama",01,015
2,28214,None,1670,None,14037,None,1463,None,1725,None,...,570,None,402,None,383,None,0500000US01043,"Cullman County, Alabama",01,043
3,19244,None,1430,None,7810,None,1106,None,1361,None,...,374,None,97,None,129,None,0500000US01049,"DeKalb County, Alabama",01,049
4,26127,None,1528,None,15754,None,1514,None,1517,None,...,402,None,125,None,147,None,0500000US01051,"Elmore County, Alabama",01,051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,36327,None,2059,None,9186,None,1443,None,355,None,...,842,None,1408,None,597,None,0500000US72113,"Ponce Municipio, Puerto Rico",72,113
844,81494,None,3483,None,38970,None,2834,None,2581,None,...,1020,None,2781,None,869,None,0500000US72127,"San Juan Municipio, Puerto Rico",72,127
845,17893,None,1331,None,8319,None,1470,None,145,None,...,181,None,273,None,281,None,0500000US72135,"Toa Alta Municipio, Puerto Rico",72,135
846,19080,None,1453,None,8356,None,1217,None,304,None,...,326,None,435,None,323,None,0500000US72137,"Toa Baja Municipio, Puerto Rico",72,137


In [7]:
housing_percent_income_2022_df = housing_percent_income_2022[['NAME', 'B25091_001E', 'B25091_002E', 'B25091_003E', 'B25091_004E', 'B25091_005E', 'B25091_006E', 'B25091_007E',
                                                              'B25091_008E', 'B25091_009E', 'B25091_010E', 'B25091_011E', 'B25091_012E',
                                                              'B25091_013E', 'B25091_014E', 'B25091_015E', 'B25091_016E', 'B25091_017E',
                                                              'B25091_018E', 'B25091_019E', 'B25091_020E', 'B25091_021E', 'B25091_022E',
                                                              'GEO_ID'
 ]]

newcol_housing_percent_income_2022 = housing_percent_income_2022_df.rename(columns={'NAME': 'County', 'B25091_001E': 'Total Owner-Occupied Units', 'B25091_002E':'Mort=Y', 'B25091_003E': 'Mort=N',
                                                            'B25091_004E': 'Mort/Income <10%', 'B25091_005E': 'Mort/Income 10-14.9%', 'B25091_006E': 'Mort/Income 15-19.9%', 'B25091_007E': 'Mort/Income 20-24.9%',
                                                            'B25091_008E': 'Mort/Income 25-29.9%', 'B25091_009E': 'Mort/Income 30-34.9%', 'B25091_010E':'Mort/Income 35-39.9%' , 'B25091_011E': 'Mort/Income 40-49.9%',
                                                            'B25091_012E': 'Mort/Income >=50%', 'B25091_013E': 'MO Costs=Y', 'B25091_014E': 'MO Costs=N', 'B25091_015E': '<$500/Mo Cost', 'B25091_016E': '$500-999/Mo Cost',
                                                            'B25091_017E': '$1000-1499/Mo Cost', 'B25091_018E': '$1500-1999/Mo Cost', 'B25091_019E': '$2000-2499/Mo Cost', 'B25091_020E': '$2500-2999/Mo Cost',
                                                            'B25091_021E': '>$3000/Mo Cost', 'B25091_022E': 'No Monthly Owner Costs'})
newcol_housing_percent_income_2022

#$___ Owner Cost = "Mortgage Status by Selected Monthly Owner Costs as a Percentage of Household Income in the Past 12 Months "

,County,Total Owner-Occupied Units,Mort=Y,Mort=N,Mort/Income <10%,Mort/Income 10-14.9%,Mort/Income 15-19.9%,Mort/Income 20-24.9%,Mort/Income 25-29.9%,Mort/Income 30-34.9%,...,MO Costs=N,<$500/Mo Cost,$500-999/Mo Cost,$1000-1499/Mo Cost,$1500-1999/Mo Cost,$2000-2499/Mo Cost,$2500-2999/Mo Cost,>$3000/Mo Cost,No Monthly Owner Costs,GEO_ID
0,"Baldwin County, Alabama",76213,45042,4140,9567,8624,6230,3314,2430,1609,...,19491,3995,2056,1359,1114,673,216,457,1071,0500000US01003
1,"Calhoun County, Alabama",32866,14709,1392,3901,3860,1388,1300,1308,170,...,9743,2742,1890,804,419,297,256,150,1037,0500000US01015
2,"Cullman County, Alabama",28214,14037,1725,3467,3028,2138,698,486,837,...,7360,2113,845,806,796,328,141,253,1133,0500000US01043
3,"DeKalb County, Alabama",19244,7810,1361,1218,1316,1401,745,293,341,...,6097,2027,1395,484,409,88,102,144,591,0500000US01049
4,"Elmore County, Alabama",26127,15754,1517,4473,3451,2816,1258,499,186,...,5976,1541,997,465,187,32,37,364,649,0500000US01051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,"Ponce Municipio, Puerto Rico",36327,9186,355,1581,1407,604,918,1200,646,...,12854,4312,2029,2001,1101,448,458,344,2186,0500000US72113
844,"San Juan Municipio, Puerto Rico",81494,38970,2581,5965,6062,5654,4078,2233,1579,...,18935,6675,4426,2451,1476,1218,549,715,3298,0500000US72127
845,"Toa Alta Municipio, Puerto Rico",17893,8319,145,1525,1661,1482,1071,851,119,...,5339,1327,1443,232,190,72,337,201,160,0500000US72135
846,"Toa Baja Municipio, Puerto Rico",19080,8356,304,1610,728,1639,900,453,342,...,5476,2351,976,303,251,187,173,40,532,0500000US72137


In [11]:
final_frame = newcol_housing_percent_income_2022[['County', 'Total Owner-Occupied Units', 'Mort=Y', 'Mort=N',
                                                            'Mort/Income <10%', 'Mort/Income 10-14.9%', 'Mort/Income 15-19.9%', 'Mort/Income 20-24.9%',
                                                            'Mort/Income 25-29.9%', 'Mort/Income 30-34.9%', 'Mort/Income 35-39.9%', 'Mort/Income 40-49.9%',
                                                            'Mort/Income >=50%', 'GEO_ID']]
final_frame

,County,Total Owner-Occupied Units,Mort=Y,Mort=N,Mort/Income <10%,Mort/Income 10-14.9%,Mort/Income 15-19.9%,Mort/Income 20-24.9%,Mort/Income 25-29.9%,Mort/Income 30-34.9%,Mort/Income 35-39.9%,Mort/Income 40-49.9%,Mort/Income >=50%,GEO_ID
0,"Baldwin County, Alabama",76213,45042,4140,9567,8624,6230,3314,2430,1609,2145,6575,408,0500000US01003
1,"Calhoun County, Alabama",32866,14709,1392,3901,3860,1388,1300,1308,170,303,926,161,0500000US01015
2,"Cullman County, Alabama",28214,14037,1725,3467,3028,2138,698,486,837,99,1515,44,0500000US01043
3,"DeKalb County, Alabama",19244,7810,1361,1218,1316,1401,745,293,341,426,661,48,0500000US01049
4,"Elmore County, Alabama",26127,15754,1517,4473,3451,2816,1258,499,186,664,849,41,0500000US01051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,"Ponce Municipio, Puerto Rico",36327,9186,355,1581,1407,604,918,1200,646,845,1608,22,0500000US72113
844,"San Juan Municipio, Puerto Rico",81494,38970,2581,5965,6062,5654,4078,2233,1579,3038,7152,628,0500000US72127
845,"Toa Alta Municipio, Puerto Rico",17893,8319,145,1525,1661,1482,1071,851,119,87,1199,179,0500000US72135
846,"Toa Baja Municipio, Puerto Rico",19080,8356,304,1610,728,1639,900,453,342,747,1633,0,0500000US72137


In [12]:
final_frame.to_csv("C:/Users/Daniel Quinn/Desktop/Bootcamp/Project_1_Working_Copy/Census/7-17-24 Work/7-18-CSVs/housing_affordability_2022.csv")

In [8]:
#This is county-level data, tracking high-cost 
#Not sure this really adds anything... Gonna ### it out

#housing_url = f"https://api.census.gov/data/2022/acs/acs1?get=NAME,B25082_001E,B25082_002E,B25082_003E&for=county:*&in=state:*&key={api_key}"
#print(housing_url)

# Make the API request
#response = requests.get(housing_url)
#data = response.json()

# Convert the JSON data into a DataFrame
#columns = data[0]
#data_rows = data[1:]
#df = pd.DataFrame(data_rows, columns=columns)

# Convert numeric columns to appropriate data types
#numeric_columns = columns[1:-2]
#df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Calculate percentages
#df['Pct_30_34.9_Mortgage'] = df['B25082_002E'] / df['B25082_001E'] * 100
#df['Pct_35Plus_Mortgage'] = df['B25082_003E'] / df['B25082_001E'] * 100

# Filter counties with high cost burden
#high_cost_burden = df[(df['Pct_30_34.9_Mortgage'] + df['Pct_35Plus_Mortgage']) > 30]
#print(high_cost_burden[['NAME', 'Pct_30_34.9_Mortgage', 'Pct_35Plus_Mortgage']])


In [9]:
#high_cost_burden = pd.DataFrame(high_cost_burden)
#high_cost_burden